In [117]:
import time
from enum import Enum

METABLOCK_HEADER_SIZE = 4 # Taille en byte du header d'un metadata block

# Différents types de metadata blocks
BLOCK_TYPE = {
    0 : "STREAMINFO",
    1 : "PADDING",
    2 : "APPLICATION",
    3 : "SEEKTABLE",
    4 : "VORBIS_COMMENT",
    5 : "CUESHEET",
    6 : "PICTURE",
    127: "INVALID"
}

# Fonction pour lire un fichier en bytes
def bytes_from_file(filename, chunksize=8192):
    with open(filename, "rb") as f:
        while True:
            chunk = f.read(chunksize)
            if chunk:
                for b in chunk:
                    yield b
            else:
                break

# Formate un nombre binaire au format byte (8 bits)
def intToByte(b):
    return "{0:08b}".format(b)

# Retourne la valeur ASCII d'un nombre binaire
def byteToASCII(b):
    n = int('0b' + b, 2)
    return n.to_bytes((n.bit_length() + 7) // 8, 'big').decode()

# Class permettant d'effectuer un benchmark
class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        if self.name:
            print('[%s]' % self.name),
        print('Elapsed: %s' % (time.time() - self.tstart))

# Lit le "flac Header". Est censé retourner fLaC
def getFlacHeader(data):
    s = ""
    for i in range(4):
        s += byteToASCII(data[i])
    return s

# Retourne un tuple (isLastMetadataBlock, metadataBlockType, metadataBlockLength)
def getMetadataBlockHeader(data, cursor):
    try:
        block_type = BLOCK_TYPE[int('0b' + data[cursor][1:8], 2)]
    except KeyError:
        block_type = "RESERVED"
    return (data[cursor][0] == "1", block_type, int('0b' + ''.join(data[cursor + 1: cursor + 4]), 2))

In [39]:
# Stocke dans la variable data le fichier flac au format binaire
with Timer("Reading Flac File"):
        filename = "01 - The Post War Dream.flac"
        data = []
        for b in bytes_from_file(filename):
            data.append(intToByte(b))

[Reading Flac File]
Elapsed: 99.46884393692017


In [127]:
metadataBlocks = []
if (getFlacHeader(data) == "fLaC"): # Si c'est un fichier flac alors on récupère les blocks
    cursor = 4
    isLast, block_type, size_block = getMetadataBlockHeader(data, cursor)
    while (not isLast):
        cursor += size_block + METABLOCK_HEADER_SIZE
        tmp = getMetadataBlockHeader(data, cursor)
        isLast = tmp[0]
        size_block = tmp[2]
        metadataBlocks.append(tmp)
    print(metadataBlocks)
else:
    print("Not a fLaC file")

[(False, 'SEEKTABLE', 324), (False, 'VORBIS_COMMENT', 383), (False, 'PICTURE', 912704), (True, 'PADDING', 4096)]


[(False, 'SEEKTABLE', 324),
 (False, 'VORBIS_COMMENT', 383),
 (False, 'PICTURE', 912704),
 (True, 'PADDING', 4096)]